In [1]:
%run ../Python_files/util_data_storage_and_load.py

In [2]:
%run ../Python_files/load_dicts.py

In [3]:
%run ../Python_files/util.py

In [4]:
# load logit_route_choice_probability_matrix
P = zload('../temp_files/logit_route_choice_probability_matrix.pkz')

In [5]:
import numpy as np
from numpy.linalg import inv

P = np.matrix(P)

In [6]:
np.size(P, 0), np.size(P, 1)

(56, 140)

In [7]:
import numpy as np
from numpy.linalg import inv

def samp_cov(x):
    """
    x: sample matrix, each column is a link flow vector sample; 24 * K
    K: number of samples
    S: sample covariance matrix
    ----------------
    return: inv(S)
    ----------------
    """
    x = np.matrix(x)
    K = np.size(x, 1)
    x_mean = sum(x[:,k] for k in range(K)) / K
    S = sum(np.dot(x[:,k] - x_mean, np.transpose(x[:,k] - x_mean)) for k in range(K)) / (K - 1)
    return S

In [8]:
# load path-link incidence matrix
A = zload('../temp_files/path-link_incidence_matrix.pkz')

In [9]:
# load link counts data

import json

with open('../temp_files/link_day_minute_Jul_dict_JSON.json', 'r') as json_file:
    link_day_minute_Jul_dict_JSON = json.load(json_file)

In [10]:
week_day_Jul_list = [2, 3, 4, 5, 6, 9, 10, 11, 12, 13, 16, 17, 18, 19, 20, 23, 24, 25, 26, 27, 30, 31]

In [11]:
link_day_minute_Jul_list = []
for link_idx in range(24):
    for day in week_day_Jul_list: 
        for minute_idx in range(120):
            key = 'link_' + str(link_idx) + '_' + str(day)
            link_day_minute_Jul_list.append(link_day_minute_Jul_dict_JSON[key] ['NT_flow_minute'][minute_idx])

In [12]:
len(link_day_minute_Jul_list)

63360

In [13]:
x = np.matrix(link_day_minute_Jul_list)
x = np.matrix.reshape(x, 24, 2640)

x = np.nan_to_num(x)
y = np.array(np.transpose(x))
y = y[np.all(y != 0, axis=1)]
x = np.transpose(y)
x = np.matrix(x)

In [14]:
np.size(x, 0), np.size(x, 1)

(24, 2536)

In [15]:
x[:,:2]

matrix([[  8641.2762886 ,   9699.88216437],
        [  9155.4101036 ,   8569.87208871],
        [ 10165.94751452,  14097.18782151],
        [ 12758.38421662,  14328.8396033 ],
        [ 13260.45046225,  14684.90411116],
        [ 10680.6785583 ,   9094.56623383],
        [ 13103.13309657,  11295.00614952],
        [  6944.68018126,   9291.67467512],
        [ 18333.56642812,  15171.06160063],
        [ 15848.0437538 ,  12327.58221841],
        [ 15579.38823825,  18216.90209321],
        [ 14544.52661243,  17231.18619011],
        [  8967.91444509,   7740.08957176],
        [ 14022.96299893,  15783.89328678],
        [ 14842.85476081,   9658.34558263],
        [  4965.31040117,   8066.19171572],
        [ 10502.40728514,   9239.40024648],
        [  9925.84183656,   8746.74420354],
        [ 13714.99412816,  14637.09856391],
        [  6384.16690799,  10483.00408518],
        [  8386.74751889,  11856.88784604],
        [ 10235.66926152,   8351.95161044],
        [ 13562.43580952,  11426

In [16]:
np.size(A,0), np.size(A,1)

(24, 140)

In [17]:
from gurobipy import *

L = 56  # dimension of lam

K = np.size(x, 1)
S = samp_cov(x)
inv_S = inv(S)

A_t = np.transpose(A)
P_t = np.transpose(P)
# PA'
PA_t = np.dot(P, A_t)
# AP_t
AP_t = np.transpose(PA_t)

Q = np.dot(np.dot(PA_t, inv_S), AP_t)
b = sum([np.dot(np.dot(PA_t, inv_S), x[:, k]) for k in range(K)])


model = Model("OD_matrix_estimation")

lam = []
for l in range(L):
    lam.append(model.addVar(name='lam_' + str(l)))

model.update() 

# Set objective: (K/2) lam' * Q * lam - b' * lam
obj = 0
for i in range(L):
    for j in range(L):
        obj += (1.0 /2) * K * lam[i] * Q[i, j] * lam[j]
for l in range(L):
    obj += - b[l] * lam[l]
model.setObjective(obj)

# Add constraint: lam >= 0
for l in range(L):
    model.addConstr(lam[l] >= 0)

model.update() 

model.optimize()

lam_list = []
for v in model.getVars():
    print('%s %g' % (v.varName, v.x))
    lam_list.append(v.x)
print('Obj: %g' % obj.getValue())

Optimize a model with 56 rows, 56 columns and 56 nonzeros
Model has 1596 quadratic objective terms
Coefficient statistics:
  Matrix range    [1e+00, 1e+00]
  Objective range [5e-03, 1e+01]
  Bounds range    [0e+00, 0e+00]
  RHS range       [0e+00, 0e+00]
Presolve removed 56 rows and 0 columns
Presolve time: 0.00s
Presolved: 0 rows, 56 columns, 0 nonzeros
Presolved model has 1596 quadratic objective terms
Ordering time: 0.00s

Barrier statistics:
 Free vars  : 24
 AA' NZ     : 2.760e+02
 Factor NZ  : 3.000e+02
 Factor Ops : 4.900e+03 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0  -2.65894169e+06  0.00000000e+00  3.70e+03 3.04e+02  9.17e+05     0s
   1   1.30241916e+06 -2.77723678e+06  3.83e+02 1.02e+02  1.64e+05     0s
   2   2.11891849e+05 -1.05946432e+06  3.83e-04 1.14e-13  2.27e+04     0s
   3  -2.05666774e+05 -3.73186382e+05  3.83e-10 5.68e-14  2.9

In [18]:
# write estimation result to file
n = 8  # number of nodes
with open('../temp_files/OD_demand_matrix_Jul_weekday_NT.txt', 'w') as the_file:
    idx = 0
    for i in range(n + 1)[1:]:
        for j in range(n + 1)[1:]:
            if i != j: 
                the_file.write("%d,%d,%f\n" %(i, j, lam_list[idx]))
                idx += 1

In [19]:
lam_list

[2.0079347975719333e-09,
 13320.875573772373,
 2.204050750578914e-09,
 1472.7740653715432,
 2849.964698580953,
 3.675820767829065e-05,
 6.533283335169606e-07,
 0.013918873957502548,
 1940.386799106939,
 11744.261203785278,
 855.8982726411656,
 922.1979426812499,
 3.675820789416978e-05,
 3.69687577270363e-05,
 12535.290635843252,
 6036.496277165886,
 65.8116751571556,
 1641.9564988257914,
 3192.30791116154,
 18902.16953391642,
 3.6769080876745215e-05,
 4219.794970089871,
 6188.586593116062,
 1912.272186739496,
 4189.662430372102,
 2647.852826460685,
 2.1074273150607705e-05,
 10623.32844307531,
 4.077923404078774,
 3.8467692337174526,
 4286.402950862192,
 7991.473950609282,
 7578.288686520654,
 2.0863034639778926e-05,
 2.0873630852208717e-05,
 5275.660684677696,
 3213.0651300885806,
 0.0003391754634356767,
 9.965188272476098,
 8045.941461679888,
 1936.1715108744595,
 1790.6957122732688,
 6.256273825279538e-06,
 6.256273778051501e-06,
 19640.796808271876,
 44.25751255189755,
 1521.1031289